In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import pandas as pd
import os

In [3]:
deprel = "det"

In [4]:
root = ".../margendatud/"+deprel
fil = root + "/" + "project-6-at-2023-04-06-20-18-ab90b718.json"
with open(fil, encoding="utf-8") as f:
    data = json.load(f)

In [5]:
fil2 = ".../train_dev_for_labelstudio/" + deprel + "/" + "ls_puupank_det_export_big_v1_fileinfo.txt"
with open(fil2, "r", encoding="utf-8") as f:
    fileinfo = f.readlines()
    

In [6]:
data[0]

{'id': 56987,
 'annotations': [{'id': 13762,
   'completed_by': 1,
   'result': [{'value': {'start': 26,
      'end': 38,
      'text': 'ühte ja sama',
      'labels': ['det_phrases']},
     'id': '9NdwKNqP4R',
     'from_name': 'label',
     'to_name': 'text',
     'type': 'labels',
     'origin': 'prediction'},
    {'value': {'choices': ['free']},
     'id': 'd6Y54eL2PU',
     'from_name': 'review',
     'to_name': 'text',
     'type': 'choices',
     'origin': 'manual'}],
   'was_cancelled': False,
   'ground_truth': False,
   'created_at': '2023-03-30T15:13:46.446461Z',
   'updated_at': '2023-03-30T15:13:46.446499Z',
   'lead_time': 18.378,
   'prediction': {'id': 56987,
    'model_version': 'undefined',
    'created_ago': '1\xa0minute',
    'result': [{'to_name': 'text',
      'from_name': 'label',
      'type': 'labels',
      'value': {'start': 26,
       'end': 38,
       'text': ['ühte', 'ja', 'sama'],
       'labels': ['det_phrases']}}],
    'score': None,
    'cluster': None

In [7]:
all_data = []

for obj in data:
    idx = obj["id"]
    removed = obj["annotations"][0]["result"][0]["value"]["text"]
    if len(obj["annotations"][0]["result"]) == 1:
        print(idx)
    else:
        verdict = obj["annotations"][0]["result"][1]["value"]["choices"][0]
        sentence = obj["data"]["text"]
        
        remove_start = obj["annotations"][0]["result"][0]["value"]["start"]
        remove_end = obj["annotations"][0]["result"][0]["value"]["end"]
        short_sent = sentence[:remove_start] + sentence[remove_end:].strip()
        
        match = False
        for line in fileinfo[1:]:
            line = line.strip()
            if sentence.replace(" ", "") in line.replace(" ", "") and removed==line.split("\\")[2]:
                fname = line.split("\\")[0]
                cons_score = line.split("\\")[3]
                ual = line.split("\\")[4]
                la = line.split("\\")[5]
                root = ".../UDpuupank/UD2_11_udreposse/"
                if os.path.isfile(root+"Train/"+fname):
                    fpath = "Train/"+fname
                elif os.path.isfile(root+"Dev/"+fname):
                    fpath = "Dev/"+fname
                    
                match = True
                
                all_data.append([idx, fpath, sentence, remove_start, remove_end, removed, verdict, short_sent, cons_score, ual, la])    
                break
        
        #if not match:
        #    print(sentence)

        #all_data.append([idx, fpath, sentence, remove_start, remove_end, removed, verdict, short_sent, cons_score, ual, la])

In [8]:
df = pd.DataFrame(all_data, columns=["id", "fpath", "sentence", "remove_start", "remove_end", "removed", "type", "short_sent", "cons_score", "ual", "la"])


In [9]:
df

,id,fpath,sentence,remove_start,remove_end,removed,type,short_sent,cons_score,ual,la
0,56987,Train/tea_AA_05_6_ud211.conllu,Et protsessorid töötlevad ühte ja sama program...,26,38,ühte ja sama,free,Et protsessorid töötlevad programmi ka samade ...,93.3,93.3,100.0
1,56988,Train/tea_AA_05_6_ud211.conllu,Et protsessorid töötlevad ühte ja sama program...,52,58,samade,bound,Et protsessorid töötlevad ühte ja sama program...,94.1,94.1,100.0
2,56989,Train/tea_AA_05_6_ud211.conllu,Et protsessorid töötlevad ühte ja sama program...,96,100,samu,bound,Et protsessorid töötlevad ühte ja sama program...,94.1,94.1,100.0
3,56990,Train/tea_dr8047.conllu,"J.A.Nettleton ( 1991 ) rõhutab , et ω-3-rasvha...",72,78,mõnede,free,"J.A.Nettleton ( 1991 ) rõhutab , et ω-3-rasvha...",100.0,100.0,100.0
4,56991,Train/aja_luup200106_osa_5_ud211.conllu,"Meil neid geene lihtsalt ei ole , ” kinnitab K...",5,9,neid,bound,"Meil geene lihtsalt ei ole , ” kinnitab Kääriä...",70.0,70.0,80.0
...,...,...,...,...,...,...,...,...,...,...,...
2755,59809,Train/tea_AA_05_6_ud211.conllu,sooduskampaaniate ning kupongide abil võimenda...,49,53,ühte,bound,sooduskampaaniate ning kupongide abil võimenda...,80.0,100.0,80.0
2756,59810,Train/aja_ml200247_osa_2_ud211.conllu,Mida rohkem tuntakse Eestit kui väikest ja tub...,125,129,neis,free,Mida rohkem tuntakse Eestit kui väikest ja tub...,95.5,95.5,100.0
2757,59811,Train/aja_ee200110_osa_3_ud211.conllu,Omanike viimistlusplaanis mahub 230 ruutmeetri...,57,61,kogu,bound,Omanike viimistlusplaanis mahub 230 ruutmeetri...,95.7,95.7,100.0
2758,59812,Train/aja_horisont2000_osa_2_ud211.conllu,Kes ikkagi annab meile siin ilmas õiguse teisi...,41,46,teisi,free,Kes ikkagi annab meile siin ilmas õiguse eluso...,92.9,92.9,100.0


In [10]:
df_free = df.loc[df['type'] == "free"]
df_free

,id,fpath,sentence,remove_start,remove_end,removed,type,short_sent,cons_score,ual,la
0,56987,Train/tea_AA_05_6_ud211.conllu,Et protsessorid töötlevad ühte ja sama program...,26,38,ühte ja sama,free,Et protsessorid töötlevad programmi ka samade ...,93.3,93.3,100.0
3,56990,Train/tea_dr8047.conllu,"J.A.Nettleton ( 1991 ) rõhutab , et ω-3-rasvha...",72,78,mõnede,free,"J.A.Nettleton ( 1991 ) rõhutab , et ω-3-rasvha...",100.0,100.0,100.0
5,56992,Dev/ilu_ruben_ud211.conllu,"Arvasin kohe , et järgmised kaheksakümmend gra...",68,73,mingi,free,"Arvasin kohe , et järgmised kaheksakümmend gra...",100.0,100.0,100.0
6,56993,Train/tea_toohoive.conllu,Kui 1998 II kvartalis moodustas koolitusfirmad...,118,125,kõigist,free,Kui 1998 II kvartalis moodustas koolitusfirmad...,None,None,None
10,56997,Train/aja_luup200106_osa_4_ud211.conllu,Nad ei ole nõus sellisest eluviisist loobuma .,16,25,sellisest,free,Nad ei ole nõus eluviisist loobuma .,100.0,100.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...
2750,59775,Train/tea_AA_05_6_ud211.conllu,Välkmälude üheks väga perspektiivseks kasutusa...,11,16,üheks,free,Välkmälude väga perspektiivseks kasutusalaks o...,100.0,100.0,100.0
2753,59807,Train/aja_luup200106_osa_2_ud211.conllu,IR : Kui teistes kandidaatriikides võivad pära...,59,65,mõnede,free,IR : Kui teistes kandidaatriikides võivad pära...,100.0,100.0,100.0
2756,59810,Train/aja_ml200247_osa_2_ud211.conllu,Mida rohkem tuntakse Eestit kui väikest ja tub...,125,129,neis,free,Mida rohkem tuntakse Eestit kui väikest ja tub...,95.5,95.5,100.0
2758,59812,Train/aja_horisont2000_osa_2_ud211.conllu,Kes ikkagi annab meile siin ilmas õiguse teisi...,41,46,teisi,free,Kes ikkagi annab meile siin ilmas õiguse eluso...,92.9,92.9,100.0


In [11]:
df_free.to_csv(f"deprel_free_analysis/csv_files/{deprel}_free.csv", sep=";", index=False)

In [12]:
df_free_cons = df_free[df_free["cons_score"]=="100.0"]
df_free_cons

,id,fpath,sentence,remove_start,remove_end,removed,type,short_sent,cons_score,ual,la
3,56990,Train/tea_dr8047.conllu,"J.A.Nettleton ( 1991 ) rõhutab , et ω-3-rasvha...",72,78,mõnede,free,"J.A.Nettleton ( 1991 ) rõhutab , et ω-3-rasvha...",100.0,100.0,100.0
5,56992,Dev/ilu_ruben_ud211.conllu,"Arvasin kohe , et järgmised kaheksakümmend gra...",68,73,mingi,free,"Arvasin kohe , et järgmised kaheksakümmend gra...",100.0,100.0,100.0
10,56997,Train/aja_luup200106_osa_4_ud211.conllu,Nad ei ole nõus sellisest eluviisist loobuma .,16,25,sellisest,free,Nad ei ole nõus eluviisist loobuma .,100.0,100.0,100.0
22,57009,Train/ilu_valton_ud211.conllu,"Kausis oli vett , aga külma vett kõht ei tahtn...",67,71,muud,free,"Kausis oli vett , aga külma vett kõht ei tahtn...",100.0,100.0,100.0
23,57010,Train/tea_AA_05_6_ud211.conllu,"Lisaks IQEU üksuse juhtimisele , mis hõlmab nä...",272,278,mõnede,free,"Lisaks IQEU üksuse juhtimisele , mis hõlmab nä...",100.0,100.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...
2731,59756,Train/aja_pm20000218_osa_5_ud211.conllu,Zhirinovskile sai saatuslikuks üks ülearune ko...,31,34,üks,free,Zhirinovskile sai saatuslikuks ülearune korter .,100.0,100.0,100.0
2740,59765,Train/aja_ee200110_osa_8_ud211.conllu,Lõhna poolest võiks see kõik olla sama hästi p...,24,28,kõik,free,Lõhna poolest võiks see olla sama hästi plasti...,100.0,100.0,100.0
2745,59770,Train/arborest_ud211.conllu,Pikk oli mööda rõdu mingisse korterisse sisse ...,20,28,mingisse,free,Pikk oli mööda rõdu korterisse sisse roninud .,100.0,100.0,100.0
2750,59775,Train/tea_AA_05_6_ud211.conllu,Välkmälude üheks väga perspektiivseks kasutusa...,11,16,üheks,free,Välkmälude väga perspektiivseks kasutusalaks o...,100.0,100.0,100.0


In [13]:
df_free_cons.to_csv(f"deprel_free_analysis/csv_files/{deprel}_free_conserved.csv", sep=";", index=False)

In [10]:
df_bound = df.loc[df['type'] == "bound"]
df_bound

,id,fpath,sentence,remove_start,remove_end,removed,type,short_sent,cons_score,ual,la
1,56988,Train/tea_AA_05_6_ud211.conllu,Et protsessorid töötlevad ühte ja sama program...,52,58,samade,bound,Et protsessorid töötlevad ühte ja sama program...,94.1,94.1,100.0
2,56989,Train/tea_AA_05_6_ud211.conllu,Et protsessorid töötlevad ühte ja sama program...,96,100,samu,bound,Et protsessorid töötlevad ühte ja sama program...,94.1,94.1,100.0
4,56991,Train/aja_luup200106_osa_5_ud211.conllu,"Meil neid geene lihtsalt ei ole , ” kinnitab K...",5,9,neid,bound,"Meil geene lihtsalt ei ole , ” kinnitab Kääriä...",70.0,70.0,80.0
7,56994,Train/tea_AA_05_6_ud211.conllu,Kui liige ei ole valmis kasutama kõiki võimalu...,33,38,kõiki,bound,"Kui liige ei ole valmis kasutama võimalusi , v...",74.2,77.4,87.1
8,56995,Train/ilu_vilep_ud211.conllu,Jan kiikas igaks juhuks salongi .,11,16,igaks,bound,Jan kiikas juhuks salongi .,100.0,100.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...
2751,59776,Train/aja_ee200110_osa_11_ud211.conllu,""" Põrgust "" pärinev "" Tango "" on võib-olla ees...",62,67,kõigi,bound,""" Põrgust "" pärinev "" Tango "" on võib-olla ees...",89.4,91.5,95.7
2752,59806,Train/aja_luup200106_osa_2_ud211.conllu,IR : Kui teistes kandidaatriikides võivad pära...,9,16,teistes,bound,IR : Kui kandidaatriikides võivad pärast liitu...,100.0,100.0,100.0
2754,59808,Train/aja_luup200106_osa_2_ud211.conllu,IR : Kui teistes kandidaatriikides võivad pära...,142,149,selline,bound,IR : Kui teistes kandidaatriikides võivad pära...,100.0,100.0,100.0
2755,59809,Train/tea_AA_05_6_ud211.conllu,sooduskampaaniate ning kupongide abil võimenda...,49,53,ühte,bound,sooduskampaaniate ning kupongide abil võimenda...,80.0,100.0,80.0


In [11]:
df_bound.to_csv(f"deprel_free_analysis/csv_files/{deprel}_bound.csv", sep=";", index=False)

In [20]:
df_comma = df.loc[df['type'] == "redundant comma"]
df_comma

,id,fpath,sentence,remove_start,remove_end,removed,type,short_sent,cons_score,ual,la
210,57197,Train/tea_AA_05_6_ud211.conllu,"Teine , ilmselt veelgi võimsama toetusega prak...",0,5,Teine,redundant comma,", ilmselt veelgi võimsama toetusega praktiline...",52.9,52.9,76.5
306,57293,Dev/aja_epl20070812_ud211.conllu,Stasi eitas alati sellise poliitika olemasolu ...,153,159,teiste,redundant comma,Stasi eitas alati sellise poliitika olemasolu ...,92.0,92.0,92.0
405,57392,Train/aja_ee200110_osa_1_ud211.conllu,"Sposato möönab , et suhtles Eestis varem ka üh...",41,53,ka ühe teise,redundant comma,"Sposato möönab , et suhtles Eestis varem , ven...",88.2,88.2,94.1
586,57573,Train/ilu_kivirahk_ud211.conllu,"Tõnu silmitses parajasti seda , kuidagi nukrak...",25,29,seda,redundant comma,"Tõnu silmitses parajasti , kuidagi nukraks muu...",100.0,100.0,100.0
1047,58034,Train/tea_toohoive.conllu,Täiendavaks ülesandeks oli Eesti tööjõu-uuring...,97,111,eelkõige nende,redundant comma,Täiendavaks ülesandeks oli Eesti tööjõu-uuring...,96.8,100.0,96.8
1149,58136,Train/aja_ml200247_osa_4_ud211.conllu,"Teistes , arenenumates riikides , kuuluvad pii...",0,7,Teistes,redundant comma,", arenenumates riikides , kuuluvad piimatööstu...",93.3,93.3,100.0
1367,58354,Train/tea_toohoive.conllu,Ärikeskkonna objektiivne komponent koosneb oma...,52,66,väga mitmetest,redundant comma,Ärikeskkonna objektiivne komponent koosneb oma...,87.5,87.5,100.0
1528,58515,Train/aja_pm19980926e_ud211.conllu,"Esimest , väiksemat ruumi sisustavad laud ja r...",0,7,Esimest,redundant comma,", väiksemat ruumi sisustavad laud ja riiul , m...",92.3,92.3,100.0
2047,59034,Train/aja_luup200106_osa_4_ud211.conllu,"Kõnekäänd ütleb , et maakera sees on veel tein...",42,47,teine,redundant comma,"Kõnekäänd ütleb , et maakera sees on veel , es...",92.9,92.9,100.0
2290,59277,Train/ilu_kivirahk_ud211.conllu,"Ta oli tulnud jala juba üle viie kilomeetri , ...",58,63,selle,redundant comma,"Ta oli tulnud jala juba üle viie kilomeetri , ...",90.0,100.0,90.0


In [21]:
df_comma.to_csv(f"deprel_free_analysis/csv_files/{deprel}_redundant_comma.csv", sep=";", index=False)

In [14]:
df_un = df.loc[df['type'] == "unnatural"]
df_un

,id,fpath,sentence,remove_start,remove_end,removed,type,short_sent,cons_score,ual,la
52,57039,Train/ilu_indrikson_ud211.conllu,"Noh , aga enne poetasin sulle selle kirja post...",30,35,selle,unnatural,"Noh , aga enne poetasin sulle kirja postkasti . """,100.0,100.0,100.0
67,57054,Dev/ilu_ruben_ud211.conllu,Kuidagi ulmelisena tundus see kõik .,30,34,kõik,unnatural,Kuidagi ulmelisena tundus see .,20.0,20.0,40.0
206,57193,Train/aja_ml200247_osa_8_ud211.conllu,""" Selliseid loenguid on kõige huvitavam kuulat...",2,11,Selliseid,unnatural,""" loenguid on kõige huvitavam kuulata , kus õp...",87.5,91.7,91.7
275,57262,Train/aja_luup200202_osa_2_ud211.conllu,Rahvuslik omapära võib siin mingit mõju avalda...,28,34,mingit,unnatural,"Rahvuslik omapära võib siin mõju avaldada , ag...",92.3,96.2,96.2
362,57349,Train/aja_ml200247_osa_8_ud211.conllu,Valdavalt lähevad lõpetajad riigitööle ning ko...,97,111,just selliseks,unnatural,Valdavalt lähevad lõpetajad riigitööle ning ko...,100.0,100.0,100.0
408,57395,Dev/ilu_ruben_ud211.conllu,"Aga siin oli hämar , see lamp siin ei valgusta...",21,24,see,unnatural,"Aga siin oli hämar , lamp siin ei valgustanud ...",90.9,90.9,100.0
444,57431,Train/aja_pm19980926e_ud211.conllu,Veini pruulimise kunst ei hääbunud ka nendel a...,35,44,ka nendel,unnatural,Veini pruulimise kunst ei hääbunud aastasadade...,93.8,93.8,100.0
449,57436,Train/ilu_vilep_ud211.conllu,"Nüüd vähemalt keegi naljavunts näitab talle , ...",62,66,kõik,unnatural,"Nüüd vähemalt keegi naljavunts näitab talle , ...",60.0,64.0,80.0
480,57467,Train/aja_luup200106_osa_4_ud211.conllu,Võetakse grupp vange kokku ja hakatakse analüü...,70,75,selle,unnatural,Võetakse grupp vange kokku ja hakatakse analüü...,89.5,89.5,89.5
678,57665,Train/ilu_remsu_ud211.conllu,Euroopas usuti sedasama mis Hiinas – meie olem...,86,89,muu,unnatural,Euroopas usuti sedasama mis Hiinas – meie olem...,81.2,81.2,81.2


In [15]:
df_un.to_csv(f"deprel_free_analysis/csv_files/{deprel}_unnatural.csv", sep=";", index=False)

In [16]:
df_dub = df.loc[df['type'] == "dubious"]
df_dub

,id,fpath,sentence,remove_start,remove_end,removed,type,short_sent,cons_score,ual,la
1004,57991,Train/aja_epl20061216_ud211.conllu,""" Kui kavad muutuvad muinsuskaitse ehk sellesa...",39,48,sellesama,dubious,""" Kui kavad muutuvad muinsuskaitse ehk kultuur...",87.5,91.7,91.7
1005,57992,Train/aja_epl20061216_ud211.conllu,""" Kui kavad muutuvad muinsuskaitse ehk sellesa...",123,128,selle,dubious,""" Kui kavad muutuvad muinsuskaitse ehk sellesa...",87.5,95.8,91.7
1276,58263,Train/aja_pm20001004_osa_1_ud211.conllu,Linnavalitsus kinnitas otsusega määrata laulul...,92,95,see,dubious,Linnavalitsus kinnitas otsusega määrata laulul...,90.0,90.0,95.0


In [17]:
df_dub.to_csv(f"deprel_free_analysis/csv_files/{deprel}_dubious.csv", sep=";", index=False)

In [18]:
df_red = df.loc[df['type'] == "other redundant punctuation"]
df_red

,id,fpath,sentence,remove_start,remove_end,removed,type,short_sent,cons_score,ual,la


In [19]:
df_red.to_csv(f"deprel_free_analysis/csv_files/{deprel}_redundant_punct.csv", sep=";", index=False)

In [ ]:
df_bound = df.loc[df['type'] == "bound"]
df_bound

df_bound.to_csv(f"deprel_free_analysis/csv_files/{deprel}_bound.csv", sep=";", index=False)

df_comma = df.loc[df['type'] == "redundant comma"]
df_comma

df_comma.to_csv(f"deprel_free_analysis/csv_files/{deprel}_redundant_comma.csv", sep=";", index=False)

df_un = df.loc[df['type'] == "unnatural"]
df_un

df_un.to_csv(f"deprel_free_analysis/csv_files/{deprel}_unnatural.csv", sep=";", index=False)

df_dub = df.loc[df['type'] == "dubious"]
df_dub

df_dub.to_csv(f"deprel_free_analysis/csv_files/{deprel}_dubious.csv", sep=";", index=False)

df_red = df.loc[df['type'] == "other redundant punctuation"]
df_red

df_red.to_csv(f"deprel_free_analysis/csv_files/{deprel}_redundant_punct.csv", sep=";", index=False)

## Vabade laienditega laused conll formaadis failidesse

In [11]:
from estnltk.converters.conll.conll_exporter import sentence_to_conll
from estnltk import Text
from estnltk.taggers import WhiteSpaceTokensTagger
from estnltk.taggers import PretokenizedTextCompoundTokensTagger
from estnltk_neural.taggers.syntax.stanza_tagger.stanza_tagger import StanzaSyntaxTagger

2023-06-14 08:27:09.223529: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-14 08:27:09.608563: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-14 08:27:10.892041: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-06-14 08:27:10.892167: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

In [12]:
model_path=".../lib/python3.9/site-packages/estnltk_neural/taggers/syntax/stanza_tagger/stanza_resources/"
input_type="morph_extended"
stanza_tagger = StanzaSyntaxTagger(input_type=input_type, input_morph_layer=input_type, add_parent_and_children=True, resources_path=model_path)

In [13]:
tokens_tagger = WhiteSpaceTokensTagger()
compound_tokens_tagger = PretokenizedTextCompoundTokensTagger()

In [14]:
from tqdm import tqdm

In [15]:
shortened_sents_data_dev = []
original_sents_data_dev = []
shortened_sents_data_train = []
original_sents_data_train = []

for row in tqdm(df_free.iterrows()):

    file = row[1][1]
    
    folder = file.split("/")[0]
    #print(folder)
    #break
    
    with open(f".../UDpuupank/UD2_11_udreposse/{file}") as f:
        conll = f.readlines()
        
    removed = row[1][5].split(" ")
    remove_start = row[1][3]
    remove_end = row[1][4]
    original_sent = row[1][2]
    sent = row[1][2].replace(" ", "")

    important_data = []

    # collect rows related to this sentence
    start = False
    for i, line in enumerate(conll):
        line = line.strip()
        mod_line = line.replace(" ", "")
        if start and not line.startswith("#"):
            important_data.append(line)
        elif start and (line=="" or line.startswith("#")):
            break    
        elif line.startswith("#") and sent in mod_line:
            start = True
            important_data.append(conll[i-1].strip())
            important_data.append(line)
            
            
    
                
            
    
    # find lines with removed words
    if len(important_data) != 0:
        
        #if "sent_id" in important_data[0]:
        #    sent_id = important_data[0].split("=")[1].strip()
        #    if sent_id in sent_dict.keys():
        if folder == "Train":
            original_sents_data_train.append(important_data)    
        elif folder == "Dev":
            original_sents_data_dev.append(important_data)
        else:
            print(f"folder {file} problem")
            break
        
        
        short_data_train = []
        short_data_dev = []
        
        short_data_pre = []
        cut_word_ids = []
        short_data_pre.append(important_data[0].strip())

        remove_count = 0
        for line in important_data[2:]:
            #print(line)

            checked = False  
            if line != "" and "sent_id =" not in line and "text =" not in line:
                added = len(line.split("\t")[1].strip() + " ")
                
                if line.split("\t")[1] in removed:
                    if len(cut_word_ids)==0 and remove_count == remove_start:
                        cut_word_ids.append(line.split("\t")[0])
                    elif len(cut_word_ids)!=0 and remove_count <= remove_end:
                        cut_word_ids.append(line.split("\t")[0])
                    checked = True
                remove_count += added
            #if not checked:    
            #    short_data.append(line+"\n")

        short_sentence = original_sent[:remove_start] + original_sent[remove_end+1:]

        txt = Text(short_sentence)
        
        tokens_tagger.tag(txt)
        compound_tokens_tagger.tag(txt)
        txt.tag_layer("morph_extended")
        stanza_tagger.tag(txt)
        res = sentence_to_conll(txt.sentences[0], txt["stanza_syntax"], udpipe=True)
        res2 = res.split("\n")
        
        
        short_data_pre.append("# cut_word_ids = " + ",".join(cut_word_ids))
        short_data_pre.append('# ud_version = 2.11')
        short_sent_text = "# text = " + " ".join([span.text for span in txt.words])
        short_data_pre.append(short_sent_text)
        
        
        #if "sent_id" in important_data[0]:
        #    sent_id = important_data[0].split("=")[1].strip()
        #    if sent_id in sent_dict.keys():
        if folder == "Train":
            short_data_train = short_data_pre + res2
            shortened_sents_data_train += [short_data_train]    
        elif folder == "Dev":
            short_data_dev = short_data_pre + res2
            shortened_sents_data_dev += [short_data_dev]
        
    #print(original_sent)
    #print(important_data)
    #print("\n",short_sent_text)
    #print(sentence_to_conll(row[1][7][:-2] + row[1][7][-2:].strip()))
    
    #print(short_data)
    
    #break
    
    
    #else:
    #    print("PROBLEEMIGA:", row[1][2])
    
#shortened_sents_data




1247it [02:59,  6.94it/s]


In [16]:
sents_to_file = 2000

left = len(shortened_sents_data_dev) % sents_to_file
files = int(len(shortened_sents_data_dev)/sents_to_file)
if left != 0:
    files += 1

for i in range(files):
    with open(f"deprel_free_analysis/Dev/{deprel}_cut.conllu", "w", encoding="utf-8") as f:
        for obj in shortened_sents_data_dev[i*sents_to_file:(i+1)*sents_to_file]:
            for line in obj:
                f.write(line)
                f.write("\n")
                
    with open(f"deprel_free_analysis/Dev/{deprel}_original.conllu", "w", encoding="utf-8") as f:
        for obj in original_sents_data_dev[i*sents_to_file:(i+1)*sents_to_file]:
            for line in obj:
                f.write(line)
                f.write("\n")
                
                
left = len(shortened_sents_data_train) % sents_to_file
files = int(len(shortened_sents_data_train)/sents_to_file)
if left != 0:
    files += 1                
for i in range(files):
    with open(f"deprel_free_analysis/Train/{deprel}_cut.conllu", "w", encoding="utf-8") as f:
        for obj in shortened_sents_data_train[i*sents_to_file:(i+1)*sents_to_file]:
            for line in obj:
                f.write(line)
                f.write("\n")
                
    with open(f"deprel_free_analysis/Train/{deprel}_original.conllu", "w", encoding="utf-8") as f:
        for obj in original_sents_data_train[i*sents_to_file:(i+1)*sents_to_file]:
            for line in obj:
                f.write(line)
                f.write("\n")